In [19]:
# Innomatics Research Labs
# Advanced GenAI Internship – Entrance Test
# Name: Pooja khape

In [4]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
import pandas as pd

users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
import sqlite3

conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [8]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [10]:
gold_revenue_by_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_revenue_by_city


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [11]:
avg_order_value_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

avg_order_value_by_cuisine


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [12]:
user_total_orders = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

users_above_1000 = user_total_orders[user_total_orders > 1000]

len(users_above_1000)


2544

In [13]:
#Create rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

#Revenue by rating range
revenue_by_rating_range = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

revenue_by_rating_range


/tmp/ipython-input-1966491808.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


,total_amount
rating_range,
4.6 – 5.0,2197030.75
3.0 – 3.5,2136772.70
4.1 – 4.5,1960326.26
3.6 – 4.0,1717494.41


In [14]:
gold_avg_order_by_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_avg_order_by_city


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [15]:
#Cuisine-wise distinct restaurant count
restaurant_count = (
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)

# Cuisine-wise total revenue
revenue_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
)

#Combine both into one table
cuisine_summary = pd.DataFrame({
    "restaurant_count": restaurant_count,
    "total_revenue": revenue_by_cuisine
})

cuisine_summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [16]:
# Total number of orders
total_orders = len(final_df)

# Orders placed by Gold members
gold_orders = len(final_df[final_df["membership"] == "Gold"])

# Percentage calculation
percentage_gold_orders = round((gold_orders / total_orders) * 100)

percentage_gold_orders


50

In [17]:
#Restaurant-wise order count and average order value
restaurant_stats = (
    final_df
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

# Filter restaurants with less than 20 orders
filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

#  Sort by highest average order value
filtered_restaurants_sorted = filtered_restaurants.sort_values(
    by="avg_order_value",
    ascending=False
)

filtered_restaurants_sorted


KeyError: 'restaurant_name'

In [18]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')

In [19]:
restaurant_stats = (
    final_df
    .groupby("name")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

filtered_restaurants_sorted = filtered_restaurants.sort_values(
    by="avg_order_value",
    ascending=False
)

filtered_restaurants_sorted


,total_orders,avg_order_value
name,,
User_2429,1,1497.42
User_889,1,1492.63
User_1843,1,1484.24
User_1882,1,1481.84
User_925,1,1476.18
...,...,...
User_2368,1,119.08
User_2583,1,117.62
User_1188,1,109.58


In [20]:
restaurant_stats = (
    final_df
    .groupby("restaurant_name_y")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

filtered_restaurants_sorted = filtered_restaurants.sort_values(
    by="avg_order_value",
    ascending=False
)

filtered_restaurants_sorted


,total_orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [21]:
revenue_by_combo = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

revenue_by_combo


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
#  order_date is datetime
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

# Create quarter column
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

# Revenue by quarter
revenue_by_quarter = (
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

revenue_by_quarter


/tmp/ipython-input-3964530731.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [23]:
# Q: How many total orders were placed by users with Gold membership?
# Filter orders placed by Gold members
gold_orders = final_df[final_df["membership"] == "Gold"]

# Count total number of orders
gold_orders_count = len(gold_orders)

gold_orders_count


4987

In [24]:
# Q: What is the total revenue (rounded to nearest integer)
# generated from orders placed in Hyderabad city?

hyderabad_orders = final_df[final_df["city"] == "Hyderabad"]
hyderabad_revenue = hyderabad_orders["total_amount"].sum()

round(hyderabad_revenue)


1889367

In [25]:
# Q: How many distinct users placed at least one order?

distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [26]:
# Q: What is the average order value (rounded to 2 decimals) for Gold members?

gold_avg_order_value = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)


np.float64(797.15)

In [27]:
# Q: How many orders were placed for restaurants with rating ≥ 4.5?

high_rating_orders = final_df[final_df["rating"] >= 4.5]
len(high_rating_orders)


3374

In [28]:
# Q: How many orders were placed in the top revenue city among Gold members only?

gold_df = final_df[final_df["membership"] == "Gold"]

top_city = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_in_top_city = len(gold_df[gold_df["city"] == top_city])

top_city, orders_in_top_city


('Chennai', 1337)

In [29]:
pd.merge(orders, users, on="user_id", how="left")


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [30]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [31]:
# Q: The total number of rows in the final merged dataset

len(final_df)


10000

In [32]:
pd.merge(orders, users, on="user_id", how="left")

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold
